In [1]:
import os

In [2]:
os.chdir("../")

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class FullModelTrainerConfig:
    root_dir: Path
    X_stacked_data_path: Path
    y_stacked_data_path: Path
    partial_model_name: str
    final_model_name: str

In [9]:
from src.StockSeer.constants import *
from src.StockSeer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_full_model_trainer_config(self) -> FullModelTrainerConfig:
        config = self.config.full_model_trainer

        create_directories([config.root_dir])

        full_model_trainer_config = FullModelTrainerConfig(
            root_dir=config.root_dir,
            X_stacked_data_path = config.X_stacked_data_path,
            y_stacked_data_path = config.y_stacked_data_path,
            partial_model_name = config.partial_model_name,
            final_model_name=config.final_model_name
        )

        return full_model_trainer_config

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

import tensorflow as tf
from src.StockSeer.logging import logger
import numpy as np

class FullModelTrainer:
    def __init__(self, config: FullModelTrainerConfig):
        self.config = config

    def train(self):
        np.random.seed(42)
        tf.random.set_seed(42)

        X = np.load(self.config.X_stacked_data_path)
        y = np.load(self.config.y_stacked_data_path)
        model = load_model(self.config.partial_model_name)

        lr_callback = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2) 
        # Train the model
        logger.info("full model training started")
        model.fit(X, y, epochs=1, batch_size=32, callbacks=[lr_callback])
        model.save(self.config.final_model_name)
        logger.info(f"full model trainned and save at {self.config.final_model_name}")
    

In [11]:
try:
    config = ConfigurationManager()
    full_model_trainer_config = config.get_full_model_trainer_config()
    full_model_trainer_config = FullModelTrainer(full_model_trainer_config)
    full_model_trainer_config.train()
except Exception as e:
    raise e

[2024-03-05 16:41:57,944: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-05 16:41:57,947: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 16:41:57,949: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-05 16:41:57,951: INFO: common: created directory at: artifacts]
[2024-03-05 16:41:57,951: INFO: common: created directory at: artifacts/full_model_trainer]


[2024-03-05 16:41:58,934: INFO: 1105721778: full model training started]
338/338 [==============================] - 30s 76ms/step - loss: 0.0017 - lr: 0.0010
[2024-03-05 16:42:29,271: INFO: 1105721778: full model trainned and save at artifacts/full_model_trainer/model.h5]


c:\Users\Krish Goyani\Desktop\DS Projects\Stock-Seer\env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
